# Drive and env

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%pip install git+https://github.com/Mo5mami/wtfml.git

  Cloning https://github.com/Mo5mami/wtfml.git to /tmp/pip-req-build-22ouadt9
  Running command git clone -q https://github.com/Mo5mami/wtfml.git /tmp/pip-req-build-22ouadt9
  Created wheel for wtfml: filename=wtfml-0.0.4-cp36-none-any.whl size=12544 sha256=958ad14b4d92c14b5e0e47eddba4000210502873fc0c407f8415e1702813e6c3
  Stored in directory: /tmp/pip-ephem-wheel-cache-w26hv0ej/wheels/5e/37/fe/2ae653f8b5b64673abba424c23de5406ac217f7c3df4aa905d
Successfully built wtfml


In [ ]:
%pip install torchaudio librosa pretrainedmodels albumentations==0.4.6

     |████████████████████████████████| 7.6MB 9.5MB/s 
     |████████████████████████████████| 61kB 8.5MB/s 
     |████████████████████████████████| 122kB 37.9MB/s 
     |████████████████████████████████| 952kB 39.0MB/s 
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-cp36-none-any.whl size=60964 sha256=ba6f3d749ddbb6e668fefe89e1b6a9d3d1016ba4c30d777d59f6639b7ae12a2d
  Stored in directory: /root/.cache/pip/wheels/69/df/63/62583c096289713f22db605aa2334de5b591d59861a02c2ecd
  Created wheel for albumentations: filename=albumentations-0.4.6-cp36-none-any.whl size=65165 sha256=fd7ca48b2b78cfb977e3996fc307aa80cadc65c3e159dc52c54e8b3884512c60
  Stored in directory: /root/.cache/pip/wheels/c7/f4/89/56d1bee5c421c36c1a951eeb4adcc32fbb82f5344c086efa14
Successfully built pretrainedmodels albumentations
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Found existing installation: albumentations 0.1.12
    

In [ ]:
!pip freeze | grep torch

torch==1.7.0+cu101
torchaudio==0.7.0
torchsummary==1.5.1
torchtext==0.3.1
torchvision==0.8.1+cu101


In [ ]:
from __future__ import print_function
import argparse
import sys
import os
import random
import librosa
from tqdm.notebook import tqdm
import scipy
import numpy as np
import pandas as pd
import torch
import torchaudio
import torchvision
from scipy.io import wavfile
import IPython.display as ipd
import torch
from torch import nn
from torch.nn import functional as F
#from utils import one_hot_embedding
from torch.autograd import Variable
from torch.utils.data import DataLoader,Dataset
from sklearn.model_selection import KFold,StratifiedKFold,StratifiedShuffleSplit
import albumentations

from albumentations.pytorch.transforms import ToTensor
from google.colab.patches import cv2_imshow
from wtfml.utils import EarlyStopping
from wtfml.engine import Engine
import pretrainedmodels
from pretrainedmodels.models import nasnetamobile
import cv2
import gc
import math
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from scipy.stats.mstats import gmean
from librosa.display import specshow
from sklearn.utils import class_weight
from torch.optim.lr_scheduler import _LRScheduler
import io


/usr/local/lib/python3.6/dist-packages/torchaudio/backend/utils.py:54: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  '"sox" backend is being deprecated. '
/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything

In [ ]:
def seed_all(seed_value):
    random.seed(seed_value) # Python
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False
num_seed=42
seed_all(num_seed)

In [ ]:
!nvidia-smi

Tue Dec  1 19:58:43 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    27W / 250W |     10MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!nvidia-smi

Tue Dec  1 19:58:43 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    27W / 250W |     10MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
torch.cuda.is_available()

True

# Utils and settings

## general settings

In [ ]:
class Config:
  device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  epochs=40
  random_state=42
  train_batchsize=4
  test_batchsize=4
  val_every=5
  print_every=20
  logdir="logs"
  DATASET_PATH="audio_files"
  DATASET2_PATH="latest_keywords"
  DATASET3_PATH="nlp_keywords"
  n_folds=10
  test_size=0.1
  lr=1.2*1e-4
  aftertrain_lr=2*1e-6
  min_lr=0.1*1e-4
  experiment_id="20-11-20_exp1"

## audio settings

In [ ]:
class AudioConfig:
    audio_length=3
    sr=44100
    fixed_sr=audio_length*sr
    hop_length = 276
    fmin = 20
    fmax = 8000
    n_mels = 64
    n_mfcc=13
    n_fft = n_mels*20
    min_seconds = 0.1
    WRAP_PAD_PROB = 0.5
    pad=450
    spec_aug_prob=0.8
    mixer_prob=0.0
    audio_crop_prob=0.5
    height=228
    width=400
    duration=3.5

## plot functions

In [ ]:
def plot_signal(signals):
    fig, axes = plt.subplots(nrows=1, ncols=1, sharex=False,
                             sharey=True, figsize=(20,5))
    axes.set_title("sig")
    axes.plot(list(signals))
    
    
def plot_signals(signals):
    fig, axes = plt.subplots(nrows=2, ncols=5, sharex=False,
                             sharey=True, figsize=(20,5))
    fig.suptitle('Time Series', size=16)
    i = 0
    for x in range(2):
        for y in range(5):
            axes[x,y].set_title(list(signals.keys())[i])
            axes[x,y].plot(list(signals.values())[i])
            axes[x,y].get_xaxis().set_visible(False)
            axes[x,y].get_yaxis().set_visible(False)
            i += 1

def plot_fft(fft):
    fig, axes = plt.subplots(nrows=2, ncols=5, sharex=False,
                             sharey=True, figsize=(20,5))
    fig.suptitle('Fourier Transforms', size=16)
    i = 0
    for x in range(2):
        for y in range(5):
            data = list(fft.values())[i]
            Y, freq = data[0], data[1]
            axes[x,y].set_title(list(fft.keys())[i])
            axes[x,y].plot(freq, Y)
            axes[x,y].get_xaxis().set_visible(False)
            axes[x,y].get_yaxis().set_visible(False)
            i += 1

def plot_fbank(fbank):
    fig, axes = plt.subplots(nrows=2, ncols=5, sharex=False,
                             sharey=True, figsize=(20,5))
    fig.suptitle('Filter Bank Coefficients', size=16)
    i = 0
    for x in range(2):
        for y in range(5):
            axes[x,y].set_title(list(fbank.keys())[i])
            axes[x,y].imshow(list(fbank.values())[i],
                    cmap='hot', interpolation='nearest')
            axes[x,y].get_xaxis().set_visible(False)
            axes[x,y].get_yaxis().set_visible(False)
            i += 1

def plot_mfccs(mfccs):
    fig, axes = plt.subplots(nrows=1, ncols=1, sharex=False,
                             sharey=True, figsize=(20,5))
    
    axes.set_title("mfcc")
    
    specshow(mfccs,x_axis='time',y_axis='mel', 
                             sr=AudioConfig.sr, hop_length=AudioConfig.hop_length,
                            fmin=AudioConfig.fmin, fmax=AudioConfig.fmax)
    plt.colorbar(format='%+2.0f dB')
    
    plt.show()
def get_plot_mfccs(mfccs):
    
    fig,ax = plt.subplots(1)
    fig.subplots_adjust(left=0,right=1,bottom=0,top=1)
    ax.axis('off')
    
    
    specshow(mfccs,x_axis="time",y_axis="mel", 
                             sr=AudioConfig.sr,hop_length=AudioConfig.hop_length,
                            fmin=AudioConfig.fmin, fmax=AudioConfig.fmax)
    ax.axis('off')
    
    image=io.BytesIO()
    fig.savefig(image,bbox_inches='tight',pad_inches=0.0)
    img=np.frombuffer(image.getvalue(), dtype='uint8')
    img = cv2.imdecode(img,cv2.IMREAD_COLOR)
    image.close()
    plt.close()
    return img



def plot_class_dist(X):
    class_dis=X.groupby("label")["fn"].count()
    fig, ax = plt.subplots()
    
    ax.set_title('Class Distribution', y=1.08)
    ax.pie(class_dis, labels=class_dis.index, autopct='%1.1f%%',
          shadow=False, startangle=90)
    ax.axis('equal')
    plt.show()


def show_melspectrogram(mels, title='Log-frequency power spectrogram'):
    librosa.display.specshow(mels, x_axis='time', y_axis='mel', 
                             sr=conf.sampling_rate, hop_length=conf.hop_length,
                            fmin=conf.fmin, fmax=conf.fmax)
    plt.colorbar(format='%+2.0f dB')
    plt.title(title)
    plt.show()


## util functions

In [ ]:
"""
Read audio from a path
"""

def read_wav(filepath):
  
  sample_rate, samples = wavfile.read(filepath)
  return sample_rate,np.array(samples)

"""
Listen to audio sample
"""
def listen(samples,sample_rate):
  return ipd.Audio(samples, rate=sample_rate)

"""
Wav loader for DatasetFolder using librosa
"""
def wav_loader(path,sr=AudioConfig.sr,fixed_sr=AudioConfig.fixed_sr):
    sample,sr=librosa.load(path,sr=sr)
    result=torch.zeros(1,fixed_sr)
    length=min(fixed_sr,len(sample))
    result[0,:length]=torch.tensor(sample[:length])
    return (result,sr)
    #return (torch.tensor([sample],),sr)

"""
Wav loader for DatasetFolder using torchaudio
"""

def torch_wav_loader(path,sr=AudioConfig.sr,fixed_sr=AudioConfig.fixed_sr):
    sample,sr=torchaudio.load_wav(path)
    result=torch.zeros(1,fixed_sr)
    length=min(fixed_sr,sample.shape[1])
    result[0,:length]=sample[0,:length]
    return (result,sr)

"""
accuracy measure
"""
def accuracy(predictions,real):
    return (predictions==real).sum()*100/len(predictions)

def enveloppe(sig,sr,threshhold):
  mask=[]
  sig=pd.Series(sig).apply(np.abs)
  sig_mean=sig.rolling(window=int(sr/10),min_periods=1,center=True).mean()
  for mean in sig_mean:
    if mean>threshhold:
      mask.append(True)
    else:  mask.append(False)
  return mask


def read_audio(file_path,top_db=60):
    min_samples = int(AudioConfig.min_seconds * AudioConfig.sr)
    y, sr = librosa.load(file_path, sr=AudioConfig.sr)
    trim_y, trim_idx = librosa.effects.trim(y,top_db=top_db,frame_length=AudioConfig.n_fft, hop_length=AudioConfig.hop_length)  # trim, top_db=default(60)

    if len(trim_y) < min_samples:
        center = (trim_idx[1] - trim_idx[0]) // 2
        left_idx = max(0, center - min_samples // 2)
        right_idx = min(len(y), center + min_samples // 2)
        trim_y = y[left_idx:right_idx]

        if len(trim_y) < min_samples:
            padding = min_samples - len(trim_y)
            offset = padding // 2
            trim_y = np.pad(trim_y, (offset, padding - offset), 'constant')

    return trim_y

def test_top_db(filepath,top_db,print_mask=True):
  sample=read_audio(filepath)
  plot_signal(sample)
  sample_test=read_audio(filepath,top_db=top_db)
  plot_signal(sample_test)
  if(print_mask):  return listen(sample_test,AudioConfig.sr)
  else : return listen(sample,AudioConfig.sr)


def audio_to_melspectrogram(audio):
    
    spectrogram = librosa.feature.melspectrogram(audio,
                                                 sr=AudioConfig.sr,
                                                 n_mels=AudioConfig.n_mels,
                                                 n_fft=AudioConfig.n_fft,
                                                 hop_length=AudioConfig.hop_length,
                                                 fmin=AudioConfig.fmin,
                                                 fmax=AudioConfig.fmax,
                                                 power=0.2
                                                 )
    
    spectogram=librosa.pcen(spectrogram * (2**31))
    spectrogram = spectrogram.astype(np.float32)
    return spectrogram

def read_as_melspectrogram(file_path,time_stretch=1.0, pitch_shift=0.0,
                           debug_display=False):
    x = read_audio(file_path)
    if time_stretch != 1.0:
        x = librosa.effects.time_stretch(x, time_stretch)

    if pitch_shift != 0.0:
        librosa.effects.pitch_shift(x, config.sampling_rate, n_steps=pitch_shift)

    
    mels = audio_to_melspectrogram(x)
    if debug_display:
        import IPython
        IPython.display.display(IPython.display.Audio(x, rate=config.sampling_rate))
        show_melspectrogram(mels)
    return (mels,AudioConfig.sr)

def mix_up(x, y):
        x = np.array(x, np.float32)
        lam = np.random.beta(1.0, 1.0)
        ori_index = np.arange(int(len(x)))
        index_array = np.arange(int(len(x)))
        np.random.shuffle(index_array)        
        
        mixed_x = lam * x[ori_index] + (1 - lam) * x[index_array]
        mixed_y = lam * y[ori_index] + (1 - lam) * y[index_array]
        
        return mixed_x, mixed_y

def oversample(dataframe):
    X,y=RandomOverSampler(random_state=42).fit_sample(dataframe, dataframe["label"])
    return pd.DataFrame(X,columns=dataframe.columns).reset_index(drop=True)

def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return torch.eye(num_classes, dtype=float)[y]

"""
function to test the enveloppe
"""
def test_enveloppe(filepath,thresh=0.0005,print_mask=True):
  sample=read_audio(filepath)
  print(sample.max())
  plot_signal(sample)
  mask=enveloppe(sample,AudioConfig.sr,thresh)
  plot_signal(sample[mask])
  if(print_mask):  return listen(sample[mask],AudioConfig.sr)
  else : return listen(sample,AudioConfig.sr)

def create_csv_dataset_from_path(dataset_path):
    classes=[classe for classe in os.listdir(dataset_path)]
    class_to_idx={classe:idx for idx,classe in enumerate(classes)}
    idx_to_class={idx:classe for idx,classe in enumerate(classes)}
    
    path=[]
    target=[]
    for classe in classes:
        class_path=os.path.join(dataset_path,classe)
        for sample in os.listdir(class_path):
            path.append(os.path.join(class_path,sample))
            target.append(classe)
    
    dataset=pd.DataFrame(data={"fn":path,"label":target})
    dataset = dataset.sample(frac=1,random_state=42).reset_index(drop=True)
    return dataset,classes

def onset_test(path):
    y=read_audio(path)
    times = librosa.times_like(audio_to_melspectrogram(y))
    onset_env = librosa.onset.onset_strength(y=y, sr=AudioConfig.sr,
                                         aggregate=np.median,
                                         n_fft=AudioConfig.n_fft,
                                        hop_length=AudioConfig.hop_length,
                                         fmax=8000, n_mels=160)
    print(onset_env.argmax())
    print(onset_env.shape)
    plt.plot(times, 1 + onset_env / onset_env.max(), alpha=0.8,
           label='Median (custom mel)')

## Transformation

### spect transformation

In [ ]:
class ToMfcc(object):
    def __call__(self,image,**kwargs):
        return audio_to_melspectrogram(image)

In [ ]:
def mono_to_color(X, mean=None, std=None, norm_max=None, norm_min=None, eps=1e-6,**kwargs):
    
    X=X.transpose(1, 0, 2)
    mean = mean or X.mean()
    std = std or X.std()
    Xstd = (X - mean) / (std + eps)
    _min, _max = Xstd.min(), Xstd.max()
    norm_max = norm_max or _max
    norm_min = norm_min or _min
    if (_max - _min) > eps:
        
        V = Xstd
        V[V < norm_min] = norm_min
        V[V > norm_max] = norm_max
        V = 255 * (V - norm_min) / (norm_max - norm_min)
        V = V.astype(np.uint8)
    else:
        
        V = np.zeros_like(Xstd, dtype=np.uint8)
    return V
  

class ToColor:
    def __init__(self,
                  mean=None,
                  std=None):
        self.mean=mean
        self.std = std
        

    def __call__(self, image,**kwargs):
        return mono_to_color(image,
                            self.mean,
                            self.std,
                            )

In [ ]:
class AudioCrop:
    def __init__(self,percentage=0.75):
        self.percentage=percentage

    def __call__(self,image,**kwargs):
        
        perc=np.random.random()*(1-self.percentage)+self.percentage
        return albumentations.RandomCrop(image.shape[0],int(image.shape[1]*perc),p=1)(image=image)["image"]


In [ ]:
class Onset:
    def __init__(self,size):
        self.size=size

    def __call__(self,image,**kwargs):
        
        onset_env = librosa.onset.onset_strength(S=image)
        argmax=onset_env.argmax()
        return albumentations.Crop(x_min=argmax-self.size//2, y_min=0, x_max=argmax+self.size//2, y_max=AudioConfig.n_mels,p=1)(image=image)["image"]


In [ ]:
class PadToSize:
    def __init__(self, size, mode='constant'):
        
        self.size = size
        self.mode = mode

    def __call__(self, image,**kwargs):
        if image.shape[1] < self.size:
            padding = self.size - image.shape[1]
            offset = padding // 2
            pad_width = ((0, 0), (offset, padding - offset))
            if self.mode == 'constant':    
                image = np.pad(image, pad_width,'constant', constant_values=0)
            else:
                image = np.pad(image, pad_width, 'wrap')
        return image


In [ ]:
class AudioPad:
    def __init__(self,percentage=0.10, mode='constant'):
        self.percentage=percentage
        self.mode=mode
    def __call__(self,image,**kwargs):
        return PadToSize(int(image.shape[1]*(self.percentage+1)),self.mode)(image=image)


In [ ]:
class ImageStack:
    def __call__(self, image,**kwargs):
        delta = librosa.feature.delta(image)
        accelerate = librosa.feature.delta(image, order=2)
        image = np.stack([image, delta, accelerate], axis=-1)
        
        image = image.astype(np.float32)
        return image
        


In [ ]:
def spec_augment(spec: np.ndarray,
                 num_mask=2,
                 freq_masking=0.15,
                 time_masking=0.20,
                 value=0):
    spec = spec.copy()
    num_mask = random.randint(1, num_mask)
    for i in range(num_mask):
        all_freqs_num, all_frames_num  = spec.shape
        freq_percentage = random.uniform(0.0, freq_masking)

        num_freqs_to_mask = int(freq_percentage * all_freqs_num)
        f0 = np.random.uniform(low=0.0, high=all_freqs_num - num_freqs_to_mask)
        f0 = int(f0)
        spec[f0:f0 + num_freqs_to_mask, :] = value

        time_percentage = random.uniform(0.0, time_masking)

        num_frames_to_mask = int(time_percentage * all_frames_num)
        t0 = np.random.uniform(low=0.0, high=all_frames_num - num_frames_to_mask)
        t0 = int(t0)
        spec[:, t0:t0 + num_frames_to_mask] = value
    return spec


class SpecAugment:
    def __init__(self,
                 num_mask=2,
                 freq_masking=0.15,
                 time_masking=0.20):
        self.num_mask = num_mask
        self.freq_masking = freq_masking
        self.time_masking = time_masking

    def __call__(self, image,**kwargs):
        return spec_augment(image,
                            self.num_mask,
                            self.freq_masking,
                            self.time_masking,
                            image.min())
        
  

### Get transformation

In [ ]:
def get_transforms(train, height,width,
                   wrap_pad_prob=0.5,
                   resize_scale=(1, 0.8),
                   resize_ratio=(1, 2.4),
                   resize_prob=0.4,
                   spec_num_mask=2,
                   spec_freq_masking=0.15,
                   spec_time_masking=0.20,
                   spec_prob=0.5):
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    if train:
      
        transforms = albumentations.Compose([
            
            
            
            albumentations.OneOf([albumentations.Lambda(PadToSize(AudioConfig.pad,mode="constant"),p=0.5),
                                  albumentations.Lambda(PadToSize(AudioConfig.pad,mode="wrap"),p=0.5),
                                    ],p=1),
            
            albumentations.Lambda(AudioCrop(percentage=0.9), p=AudioConfig.audio_crop_prob),
            
            albumentations.RandomResizedCrop(height,width,scale=resize_scale, ratio=resize_ratio,p=0.0),
            
            albumentations.Resize(height,width,p=1),
            albumentations.OneOf([albumentations.Lambda(SpecAugment(num_mask=2,freq_masking=0.10,time_masking=0.16)),
                            
                                  ],p=AudioConfig.spec_aug_prob),
                                  
            
            albumentations.Lambda(ImageStack(),p=1),
            albumentations.Lambda(ToColor(),p=1),
            albumentations.Normalize (mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0),
            albumentations.pytorch.transforms.ToTensor(),
            
            
        ])
    else:
        transforms = albumentations.Compose([
            albumentations.Lambda(PadToSize(AudioConfig.pad,mode="wrap"),p=1),
            
            albumentations.CenterCrop(AudioConfig.n_mels,width,p=1),
            albumentations.Resize(height,width,p=1),
            albumentations.Lambda(ImageStack(),p=1),
            albumentations.Lambda(ToColor(),p=1),
            albumentations.Normalize (mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0),
            albumentations.pytorch.transforms.ToTensor(),
            
        ])
    return transforms

## Mixers

In [ ]:
def get_random_sample(dataset):
    rnd_idx = random.randint(0, len(dataset) - 1)
    rnd_audio,rnd_target=dataset.tensor_dict[df.loc[rnd_idx,dataset.path_col]]
    rnd_target=dataset.class_to_idx[rnd_target]
    
    rnd_audio = dataset.transform(image=rnd_audio[0])["image"]
    rnd_target=dataset.target_transform(rnd_target,num_classes=len(dataset.classes))
    return rnd_audio, rnd_target

class AddMixer:
    def __init__(self, alpha_dist='uniform'):
        assert alpha_dist in ['uniform', 'beta']
        self.alpha_dist = alpha_dist

    def sample_alpha(self):
        if self.alpha_dist == 'uniform':
            return random.uniform(0, 0.5)
        elif self.alpha_dist == 'beta':
            return np.random.beta(0.4, 0.4)

    def __call__(self, dataset, image, target):
        rnd_image, rnd_target = get_random_sample(dataset)
        alpha = self.sample_alpha()
        image = (1 - alpha) * image + alpha * rnd_image
        target = (1 - alpha) * target + alpha * rnd_target
        return image, target


class SigmoidConcatMixer:
    def __init__(self, sigmoid_range=(3, 12)):
        self.sigmoid_range = sigmoid_range

    def sample_mask(self, size):
        x_radius = random.randint(*self.sigmoid_range)

        step = (x_radius * 2) / size[1]
        x = np.arange(-x_radius, x_radius, step=step)
        y = torch.sigmoid(torch.from_numpy(x)).numpy()
        mix_mask = np.tile(y, (size[0], 1))
        return torch.from_numpy(mix_mask.astype(np.float32))

    def __call__(self, dataset, image, target):
        rnd_image, rnd_target = get_random_sample(dataset)

        mix_mask = self.sample_mask(image.shape[-2:])
        rnd_mix_mask = 1 - mix_mask

        image = mix_mask * image + rnd_mix_mask * rnd_image
        target = target + rnd_target
        target = np.clip(target, 0.0, 1.0)
        return image, target


class RandomMixer:
    def __init__(self, mixers, p=None):
        self.mixers = mixers
        self.p = p

    def __call__(self, dataset, image, target):
        mixer = np.random.choice(self.mixers, p=self.p)
        image, target = mixer(dataset, image, target)
        return image, target


class UseMixerWithProb:
    def __init__(self, mixer, prob=.5):
        self.mixer = mixer
        self.prob = prob

    def __call__(self, dataset, image, target):
        if random.random() < self.prob:
            return self.mixer(dataset, image, target)
            print(image.shape,target.shape)
        return image, target


# Load Data

In [ ]:
!cp "/content/drive/My Drive/kaggle competitions/GIZ_NLP/audio_files.zip" . >>/dev/null
!unzip audio_files.zip >>/dev/null

In [ ]:
!cp "/content/drive/My Drive/kaggle competitions/GIZ_NLP/AdditionalUtterances.zip" . >>/dev/null
!unzip AdditionalUtterances.zip >>/dev/null

In [ ]:
!cp "/content/drive/My Drive/kaggle competitions/GIZ_NLP/nlp_keywords_29Oct2020.zip" . >>/dev/null
!unzip nlp_keywords_29Oct2020.zip >>/dev/null
!rm 'nlp_keywords/.DS_Store'

In [ ]:
df1=pd.read_csv("Train.csv")
submission=pd.read_csv("SampleSubmission.csv")
submission["label"]="akawuka"
df2,_=create_csv_dataset_from_path(Config.DATASET2_PATH)
df3,_=create_csv_dataset_from_path(Config.DATASET3_PATH)
df=pd.concat([df1,df2,df3],ignore_index=True).reset_index(drop=True)
#df=df1
df_all=pd.concat([df1,df2,df3],ignore_index=True).reset_index(drop=True)

In [ ]:
print("Files in the dataset : ",len(os.listdir(Config.DATASET_PATH)))
print("train1 shape : ",df1.shape)
print("train2 shape : ",df2.shape)
print("train3 shape : ",df3.shape)
print("train all shape : ",df.shape)
print("test shape : ",submission.shape)

Files in the dataset :  2126
train1 shape :  (1109, 2)
train2 shape :  (1740, 2)
train3 shape :  (1860, 2)
train all shape :  (4709, 2)
test shape :  (1017, 195)


In [ ]:
classes=df["label"].unique()
class_to_idx={classe:idx for idx,classe in enumerate(classes)}
idx_to_class={idx:classe for idx,classe in enumerate(classes)}

# Dataset definition and loading data

In [ ]:
class CSVDataset(Dataset):
    def __init__(self, df, loader,classes=None, transform=None,
                 target_transform=None,device=torch.device("cpu")):
        super(Dataset, self).__init__()
        
        self.df=df.reset_index(drop=True)
        self.loader=loader
        self.transform=transform
        
        self.target_transform=target_transform
        self.device=device
        self.loaded=False
        self.loaded_samples=[]
        self.path_col="fn"
        self.target_col="label"
        if classes is None:
            self.classes=df[self.target_col].unique()
        else :
            self.classes=classes
        
        self.class_to_idx={classe:idx for idx,classe in enumerate(self.classes)}
        self.idx_to_class={idx:classe for idx,classe in enumerate(self.classes)}
        
    
    def load_data(self):
        self.loaded_samples=[]
        for ind in tqdm(range(len(self.df)),0):
            path=self.df.loc[ind,self.path_col]
            target=self.df.loc[ind,self.target_col]
            sample = self.loader(path)
            self.loaded_samples.append([sample,target])
        self.loaded=True
        
    def save_tensor(self,path):
        assert self.loaded==True
        torch.save(self.loaded_samples,path)
    
    def __getitem__(self, index):
        """
        Args:
            index (int): Index

        Returns:
            dict {"audio" "sample_rate" "target"}
        """
        if self.loaded:
            sample, target = self.loaded_samples[index]
        
        else:    
            path=self.df.loc[index,self.path_col]
            target=self.df.loc[index,self.target_col]
            sample = self.loader(path)

        audio=sample[0]
        sample_rate=sample[1]
        target=self.class_to_idx[target]
        
        if self.transform is not None:
            audio = self.transform(image=audio)["image"]

        if self.target_transform is not None:
            target = self.target_transform(target)

        
        

        
        return {"audio":audio,"sample_rate":sample_rate ,"target":target}
    
    def __len__(self):
        return self.df.shape[0]



In [ ]:
class PathDataset(Dataset):

    def __init__(self, df, tensor_dict,classes=None, transform=None,
                  target_transform=None,mixer=None,device=torch.device("cpu")):
          super(PathDataset, self).__init__()
          
          self.df=df.reset_index(drop=True)
          self.tensor_dict=tensor_dict
          self.transform=transform
          self.mixer=mixer
          self.target_transform=target_transform
          self.device=device
          self.path_col="fn"
          self.target_col="label"
          if classes is None:
              self.classes=df[self.target_col].unique()
          else :
              self.classes=classes
          
          self.class_to_idx={classe:idx for idx,classe in enumerate(self.classes)}
          self.idx_to_class={idx:classe for idx,classe in enumerate(self.classes)}

    def __getitem__(self, index):
          """
          Args:
              index (int): Index

          Returns:
              dict {"audio" "sample_rate" "target"}
          """
          
          path=self.df.loc[index,self.path_col]
          
          sample,target = self.tensor_dict[path]
          audio=sample[0]
          sample_rate=sample[1]
          target=class_to_idx[target]
          
          if self.transform is not None:
              audio = self.transform(image=audio)["image"]

          if self.target_transform is not None:
              target = self.target_transform(target,num_classes=len(self.classes))

          
          if self.mixer is not None:
              audio, target = self.mixer(self, audio, target)

          
          return {"audio":audio,"sample_rate":sample_rate ,"target":target}


    def __len__(self):
        return self.df.shape[0]

In [ ]:
train_transform = get_transforms(train=True,height=AudioConfig.height,
                                     width=AudioConfig.width,
                                     wrap_pad_prob=AudioConfig.WRAP_PAD_PROB)

In [ ]:
amixer = RandomMixer([
        SigmoidConcatMixer(sigmoid_range=(3, 12)),
        AddMixer(alpha_dist='uniform')
    ], p=[0.6, 0.4])
amixer = UseMixerWithProb(amixer, prob=0.0)

In [ ]:
audio_set=CSVDataset(df_all,read_as_melspectrogram,classes=classes,transform=train_transform)


In [ ]:
submission_set=CSVDataset(submission,read_as_melspectrogram,classes=classes,transform=train_transform)

In [ ]:
audio_set.load_data()

In [ ]:
tensor_dict={path:audio_set.loaded_samples[idx] for idx,path in enumerate(df_all[audio_set.path_col])}

In [ ]:
loaded_set=PathDataset(df_all,tensor_dict,classes=classes,transform=train_transform,)

In [ ]:
loaded_set.__getitem__(457)["audio"].shape

torch.Size([3, 400, 228])

In [ ]:
submission_set.load_data()

In [ ]:
submission_tensor_dict={path:submission_set.loaded_samples[idx] for idx,path in enumerate(submission[submission_set.path_col])}

# Class weights

In [ ]:
df1["num_label"]=df1.label.apply(lambda x:class_to_idx[x])

In [ ]:
class_weights = class_weight.compute_class_weight('balanced',
                                                 classes,
                                                 df1.label)
class_weights

array([0.5746114 , 1.4365285 , 0.63845711, 0.95768566, 1.1492228 ,
       0.82087343, 0.71826425, 0.95768566, 1.1492228 , 0.95768566,
       0.63845711, 0.47884283, 1.4365285 , 0.82087343, 0.71826425,
       1.4365285 , 0.63845711, 0.522374  , 1.1492228 , 1.91537133,
       0.522374  , 1.1492228 , 0.522374  , 0.82087343, 0.5746114 ,
       0.5746114 , 1.4365285 , 1.1492228 , 0.95768566, 0.95768566,
       0.95768566, 0.95768566, 1.91537133, 1.1492228 , 0.95768566,
       0.71826425, 0.82087343, 0.95768566, 0.82087343, 1.1492228 ,
       0.95768566, 0.82087343, 1.4365285 , 1.91537133, 0.82087343,
       1.4365285 , 0.82087343, 1.1492228 , 0.522374  , 1.4365285 ,
       0.95768566, 0.522374  , 0.95768566, 0.5746114 , 1.4365285 ,
       0.95768566, 1.4365285 , 0.71826425, 1.1492228 , 0.95768566,
       0.82087343, 1.1492228 , 1.4365285 , 1.1492228 , 0.63845711,
       0.82087343, 0.63845711, 1.1492228 , 1.4365285 , 0.522374  ,
       0.71826425, 0.95768566, 1.91537133, 1.1492228 , 1.14922

# Folds

In [ ]:
df["folds"]=-1
df2["folds"]=-1
df3["folds"]=-1

kf = StratifiedKFold(n_splits=Config.n_folds, random_state=Config.random_state, shuffle=True)


for fold, (_, val_index) in enumerate(kf.split(df,df["label"])):
        df.loc[val_index, "folds"] = fold

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=10.
  % (min_groups, self.n_splits)), UserWarning)


# Model

In [ ]:
class Net(torch.nn.Module):
    def __init__(self,arch,num_classes ,pretrained='imagenet'):
        super(Net, self).__init__()
        self.base_model = pretrainedmodels.__dict__[
            arch
        ](pretrained=pretrained)
        
        
        
        if arch=="dpn98":
            self.l0 = torch.nn.Linear(2688, num_classes)
        elif arch=="se_resnext50_32x4d" or arch=="resnet101" :
            self.l0 = torch.nn.Linear(2048, num_classes)
        elif arch=="dpn68":
            self.l0 = torch.nn.Linear(832, num_classes)
        elif arch=="resnet18":
            self.l0 = torch.nn.Linear(512, num_classes)
            

        elif arch=="vgg19":
            self.l0 = torch.nn.Linear(512, num_classes)
            

        elif arch=="se_resnet50":    
            self.l0 = torch.nn.Linear(2048, num_classes)
        elif arch=="resnet50":    
            self.l0 = torch.nn.Linear(2048, num_classes)  
        elif arch=="senet154":
            self.l0 = torch.nn.Linear(2048, num_classes)
        elif arch=="se_resnext101_32x4d":
            self.l0 = torch.nn.Linear(2048, num_classes)
        elif arch=="dpn107":
            self.l0 = torch.nn.Linear(2688, num_classes)
        elif arch=="densenet121":
            self.l0 = torch.nn.Linear(1024, num_classes)
            fc_size = self.base_model.last_linear.in_features
            
            self.base_model.last_linear = nn.Sequential(nn.Linear(7168, 193))


        else :
            self.l0 = torch.nn.Linear(4098, num_classes)
    def forward(self, audio, target,sample_rate):
        batch_size, _, _, _ = audio.shape
        x=audio
        x = self.base_model.features(x)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        out = self.l0(x)
        loss = torch.nn.CrossEntropyLoss()(out, torch.argmax(target, dim=1))
        return out, loss

# Training

In [ ]:
model_name="resnet18"
pretrained="imagenet"

In [ ]:
model = Net(model_name,num_classes=len(classes),pretrained=pretrained)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [ ]:
def train(fold):
    seed_all(num_seed)
    model_path=f"model_fold_{fold}.bin"
    df_train = df[df["folds"] != fold].reset_index(drop=True)
    df_valid = df[df["folds"] == fold].reset_index(drop=True)
    df_train["weights"]=df_train["label"].apply(lambda x:class_weights[class_to_idx[x]])
    print("-------------",df_train.shape,"---------------",df_valid.shape,"-------------")
    train_transfrom = get_transforms(train=True,
                                     height=AudioConfig.height,
                                     width=AudioConfig.width,
                                     wrap_pad_prob=AudioConfig.WRAP_PAD_PROB)
    valid_transfrom = get_transforms(train=False,
                                     height=AudioConfig.height,
                                     width=AudioConfig.width,
                                     wrap_pad_prob=AudioConfig.WRAP_PAD_PROB)
    mixer = RandomMixer([
        AddMixer(alpha_dist='uniform')
    ], p=[1])
    mixer = UseMixerWithProb(mixer, prob=AudioConfig.mixer_prob)
    train_dataset =PathDataset(df_train,tensor_dict,classes=classes,transform=train_transfrom,target_transform=to_categorical,mixer=mixer)
    
    
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=Config.train_batchsize, shuffle=True, num_workers=8
    )
    
    valid_dataset =PathDataset(df_valid,tensor_dict,classes=classes,transform=valid_transfrom,target_transform=to_categorical)
    
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=Config.test_batchsize, shuffle=False, num_workers=8
    )
    model = Net(model_name,num_classes=len(classes),pretrained=pretrained)
    
    model.to(Config.device)
    model.train()
    optimizer = torch.optim.AdamW(model.parameters(), lr=Config.lr)
    
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=0, factor=0.6,min_lr=Config.min_lr,verbose=True)
    

    es = EarlyStopping(patience=8, mode="min")
    eng = Engine(model, optimizer, device=Config.device)
    for epoch in range(Config.epochs):
        train_loss = eng.train(train_loader)
        valid_loss,predictions = eng.evaluate(valid_loader, return_predictions=True)
        with open('out.txt', 'a') as f:
            f.write(f"Fold = {fold}  Epoch = {epoch}, valid loss = {valid_loss}\n")
        
        scheduler.step(valid_loss)
        es(valid_loss, model, model_path=f"model_fold_{fold}.bin")
        if es.early_stop:
            print("Early stopping")
            break
    


In [ ]:
def after_train(fold):
    seed_all(num_seed)
    model_path=f"model_fold_{fold}.bin"
    model_save_path=f"model_fold_{fold}.bin"
    df_train = df[df["folds"] != fold].reset_index(drop=True)
    
    df_valid = df[df["folds"] == fold].reset_index(drop=True)
    df_train["weights"]=df_train["label"].apply(lambda x:class_weights[class_to_idx[x]])
    
    print("-------------",df_train.shape,"---------------",df_valid.shape,"-------------")
    train_transfrom = get_transforms(train=True,
                                     height=AudioConfig.height,
                                     width=AudioConfig.width,
                                     wrap_pad_prob=AudioConfig.WRAP_PAD_PROB)
    valid_transfrom = get_transforms(train=False,
                                     height=AudioConfig.height,
                                     width=AudioConfig.width,
                                     wrap_pad_prob=AudioConfig.WRAP_PAD_PROB)
    mixer = RandomMixer([
        
        AddMixer(alpha_dist='uniform')
    ], p=[1])
    mixer = UseMixerWithProb(mixer, prob=AudioConfig.mixer_prob)
    train_dataset =PathDataset(df_train,tensor_dict,classes=classes,transform=train_transfrom,target_transform=to_categorical,mixer=mixer)
    
    
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=Config.train_batchsize, shuffle=True, num_workers=8
    )
    
    
    valid_dataset =PathDataset(df_valid,tensor_dict,classes=classes,transform=valid_transfrom,target_transform=to_categorical)
    
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=Config.test_batchsize, shuffle=False, num_workers=8
    )
    model = Net(model_name,num_classes=len(classes),pretrained=pretrained)
    model.load_state_dict(torch.load(model_path))
    model.to(Config.device)
    model.train()
    optimizer = torch.optim.AdamW(model.parameters(), lr=Config.aftertrain_lr)
    
    scheduler=torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 10, eta_min=0.2*1e-6, last_epoch=-1,)
    
    es = EarlyStopping(patience=6, mode="min")
    eng = Engine(model, optimizer,scheduler=scheduler, device=Config.device)
    for epoch in range(Config.epochs):
        if epoch!=0:
            train_loss = eng.train(train_loader)
        valid_loss,predictions = eng.evaluate(valid_loader, return_predictions=True)
        
        with open('out.txt', 'a') as f:
            f.write(f"Fold = {fold}  Epoch = {epoch}, valid loss = {valid_loss}\n")
        
        
        es(valid_loss, model, model_path=model_save_path)
        if es.early_stop:
            print("Early stopping")
            break
    


In [ ]:
def eval_train(fold):
    seed_all(num_seed)
    model_path=f"model_fold_{fold}.bin"
    df_train = df[df["folds"] != fold].reset_index(drop=True)
    df_valid = df[df["folds"] == fold].reset_index(drop=True)
    
    train_transfrom = get_transforms(train=True,
                                     height=AudioConfig.height,
                                     width=AudioConfig.width,
                                     wrap_pad_prob=AudioConfig.WRAP_PAD_PROB)
    valid_transfrom = get_transforms(train=False,
                                     height=AudioConfig.height,
                                     width=AudioConfig.width,
                                     wrap_pad_prob=AudioConfig.WRAP_PAD_PROB)
    mixer = RandomMixer([
        
        AddMixer(alpha_dist='uniform')
    ], p=[1])
    mixer = UseMixerWithProb(mixer, prob=AudioConfig.mixer_prob)
    train_dataset =PathDataset(df_train,tensor_dict,classes=classes,transform=train_transfrom,target_transform=to_categorical,mixer=mixer)
    
    
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=Config.train_batchsize, shuffle=True, num_workers=8
    )

    valid_dataset =PathDataset(df_valid,tensor_dict,classes=classes,transform=valid_transfrom,target_transform=to_categorical)
    
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=Config.test_batchsize, shuffle=False, num_workers=8
    )
    model = Net(model_name,num_classes=len(classes),pretrained=pretrained)
    model.load_state_dict(torch.load(model_path))
    model.to(Config.device)
    model.train()
    optimizer = torch.optim.AdamW(model.parameters(), lr=Config.lr)
    eng = Engine(model, optimizer, device=Config.device)
    
    train_loss,predictions = eng.evaluate(train_loader, return_predictions=True)
    valid_loss,predictions = eng.evaluate(valid_loader, return_predictions=True)
    
    
    print(f"train loss = {train_loss}, valid loss = {valid_loss} ")
    return train_loss,valid_loss

    


In [ ]:
def predict(fold):
    seed_all(num_seed)
    model_path=f"model_fold_{fold}.bin"
    test_transfrom = get_transforms(train=False,
                                     height=AudioConfig.height,
                                     width=AudioConfig.width,
                                     wrap_pad_prob=AudioConfig.WRAP_PAD_PROB)
    train_transfrom = get_transforms(train=True,
                                     height=AudioConfig.height,
                                     width=AudioConfig.width,
                                     wrap_pad_prob=AudioConfig.WRAP_PAD_PROB)
    
    
    test_dataset =PathDataset(submission,submission_tensor_dict,classes=classes,transform=test_transfrom,target_transform=to_categorical)
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=Config.test_batchsize, shuffle=False, num_workers=8
    )

    model = Net(model_name,num_classes=len(classes),pretrained=pretrained)
    model.load_state_dict(torch.load(model_path))
    model.to(Config.device)
    model.eval()
    optimizer = torch.optim.Adam(model.parameters(), lr=Config.lr)
    eng = Engine(model, optimizer, device=Config.device)
    predictions = eng.predict(test_loader)
    #valid_loss,predictions = eng.evaluate(test_loader, return_predictions=True)
    predictions=torch.nn.Softmax(dim=1)(torch.cat(predictions))
    return predictions

In [ ]:
def predict_tta(fold):
    seed_all(num_seed)
    model_path=f"model_fold_{fold}.bin"
    test_transfrom = get_transforms(train=False,
                                     height=AudioConfig.height,
                                     width=AudioConfig.width,
                                     wrap_pad_prob=AudioConfig.WRAP_PAD_PROB)
    train_transfrom = get_transforms(train=True,
                                     height=AudioConfig.height,
                                     width=AudioConfig.width,
                                     wrap_pad_prob=AudioConfig.WRAP_PAD_PROB)
    
    
    test_dataset =PathDataset(submission,submission_tensor_dict,classes=classes,transform=train_transfrom,target_transform=to_categorical)
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=Config.test_batchsize, shuffle=False, num_workers=8
    )

    model = Net(model_name,num_classes=len(classes),pretrained=pretrained)
    model.load_state_dict(torch.load(model_path))
    model.to(Config.device)
    model.eval()
    optimizer = torch.optim.AdamW(model.parameters(), lr=Config.lr)
    eng = Engine(model, optimizer, device=Config.device)
    all_predictions=[]
    for i in range(30):
        all_predictions.append(torch.nn.Softmax(dim=1)(torch.cat(eng.predict(test_loader))).numpy())
    
    predictions=gmean(all_predictions)
    
    return predictions

In [ ]:
def generate_submission_csv(fold):
    seed_all(num_seed)
    model_path=f"model_fold_{fold}.bin"
    df_train = df[df["folds"] != fold].reset_index(drop=True)
    df_valid = df[df["folds"] == fold].reset_index(drop=True)
    
    train_transfrom = get_transforms(train=True,
                                     height=AudioConfig.height,
                                     width=AudioConfig.width,
                                     wrap_pad_prob=AudioConfig.WRAP_PAD_PROB)
    valid_transfrom = get_transforms(train=False,
                                     height=AudioConfig.height,
                                     width=AudioConfig.width,
                                     wrap_pad_prob=AudioConfig.WRAP_PAD_PROB)
    mixer = RandomMixer([
        
        AddMixer(alpha_dist='uniform')
    ], p=[1])
    mixer = UseMixerWithProb(mixer, prob=AudioConfig.mixer_prob)
    train_dataset =PathDataset(df_train,tensor_dict,classes=classes,transform=train_transfrom,target_transform=to_categorical,mixer=mixer)
    
    
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=Config.train_batchsize, shuffle=True, num_workers=8
    )

    valid_dataset =PathDataset(df_valid,tensor_dict,classes=classes,transform=valid_transfrom,target_transform=to_categorical)
    
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=Config.test_batchsize, shuffle=False, num_workers=8
    )
    model = Net(model_name,num_classes=len(classes),pretrained=pretrained)
    model.load_state_dict(torch.load(model_path))
    model.to(Config.device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=Config.lr)
    eng = Engine(model, optimizer, device=Config.device)
    
    
    predictions = eng.predict(valid_loader)
    
    predictions=torch.nn.Softmax(dim=1)(torch.cat(predictions))
    sample = df[df["folds"] == fold].reset_index(drop=True)
    
    sample.loc[:, classes] = predictions
    return sample

    


11:54

### Training folds

In [ ]:
%%capture
for fold in range(0,Config.n_folds):
    print("Fold : ",fold)
    train(fold)
    after_train(fold)


Buffered data was truncated after reaching the output size limit.

In [ ]:
train_losses=[]
valid_losses=[]
for fold in range(Config.n_folds):
    train_loss,valid_loss=eval_train(fold)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)


train loss = 0.09495743758353747, valid loss = 0.6202142954526777 



train loss = 0.09659368704493008, valid loss = 0.3911043093578004 



train loss = 0.0897328405242695, valid loss = 0.49899759575289526 



train loss = 0.06754315910235026, valid loss = 0.3659040279443755 



train loss = 0.0876363061122284, valid loss = 0.3790180476851489 



train loss = 0.323479715663004, valid loss = 0.66794606851329 



train loss = 0.07369889755984058, valid loss = 0.5461316176194903 



train loss = 0.08898564089229798, valid loss = 0.4097290407882025 



train loss = 0.150060227658249, valid loss = 0.645256671358997 



train loss = 0.08962246341693829, valid loss = 0.49708985211919615 


In [ ]:
np.mean(valid_losses)

0.5021391526592074

In [ ]:
p=[]
for fold in range(Config.n_folds):
    p.append(predict(fold).numpy())

In [ ]:
predictions=gmean(p,axis=0)

In [ ]:
predictions.shape

(1017, 193)

In [ ]:

prediction_file=f"resnet18_pcen.csv"
sample = pd.read_csv("SampleSubmission.csv")
sample.loc[:, classes] = predictions
sample.to_csv(prediction_file, index=False)
